In [23]:
import sys
from itertools import chain
from azureml import core
import importlib_metadata
from rsidatasciencetools.config.baseconfig import YmlConfig

In [2]:
base_imports = [
    'pandas', 
    'numpy', 
    'matplotlib',
    'scikit-learn',
    'packaging',
    'pip',
    {'pip': ['pip', 'mlflow', 'azureml',
        'azureml-core', 'azureml-mlflow', 
        'azureml-defaults',
        'azureml-contrib-services',
        'azureml-contrib-dataset',
        'azureml-mlflow', 'inference-schema',
        'scikit-learn-intelex']}
]

In [3]:
yaml_config = YmlConfig('./train-in-azure.yml')
yaml_config

YmlConfig | {
   name:train-on-amlcompute
   model_name:sklearn-ridge-regr-diabetes
   env_name:sklearn-pandas-x
   service_name:sklearn-test-diabetes
   local:['example_imports', '../../../dstools']
   dependencies:['scikit-learn', 'pandas', 'numpy', 'matplotlib', 'packaging', 'pip', {'pip': ['mlflow', 'azureml', 'azureml-core', 'azureml-defaults', 'azureml-contrib-services', 'azureml-contrib-dataset', 'azureml-mlflow', 'inference-schema', 'scikit-learn-intelex', 'sqlalchemy', 'pymssql']}] }

In [4]:
conda_pkgs = set(el for el in base_imports 
    if not(isinstance(el,dict)))
conda_pkgs.update(set(dep for dep in yaml_config['dependencies'] 
    if not(isinstance(dep,dict))))
pip_pkgs = [el for el in base_imports if isinstance(el,dict) and 'pip' in el]
if len(pip_pkgs):
    pip_pkgs = set(pip_pkgs[0]['pip'])
_pip = [dep for dep in yaml_config['dependencies'] 
    if isinstance(dep,dict) and 'pip' in dep]
if len(_pip):
    pip_pkgs.update(set(_pip[0]['pip']))

pkgs = [el for el in importlib_metadata.distributions() 
    if el.name in conda_pkgs or el.name in pip_pkgs]
pkg_versions = {'azureml-core': core.VERSION}
pkg_versions.update({lib.name: lib.version for lib in pkgs})
conda_pkgs = list(sorted(conda_pkgs))
pip_pkgs = list(sorted(pip_pkgs))

In [5]:
ws = core.Workspace.from_config('./azure-config.json')
print(ws)

Workspace.create(name='nroseveare-workspace', subscription_id='3fa70909-0b38-420d-a9ff-e0bc32420128', resource_group='nroseveare-rg')


In [6]:
# whl_pkgs = ["../../../DataScienceTools/dist/DataScienceTools-0.0.1-py2.py3-none-any.whl"]
whl_pkgs = ["../../../dist/DataScienceTools-0.0.1-py2.py3-none-any.whl"]

In [25]:
PYTHON_VERSION = "{major}.{minor}.{micro}".format(
    major=sys.version_info.major,
    minor=sys.version_info.minor,
    micro=sys.version_info.micro)
if True:
    print('conda dep: ', conda_pkgs)
    print('pip dep: ', pip_pkgs)
    
conda_dependencies = core.conda_dependencies.CondaDependencies.create(
    python_version=PYTHON_VERSION,
    conda_packages=(conda_pkgs if len(conda_pkgs) else None),
    pip_packages=(pip_pkgs if len(pip_pkgs) else None)
    )

for whl in whl_pkgs:
    whl_url = core.Environment.add_private_pip_wheel(workspace=ws,file_path=whl,exist_ok=True)
    conda_dependencies.add_pip_package(whl_url)
conda_dependencies.__dict__

conda dep:  ['matplotlib', 'numpy', 'packaging', 'pandas', 'pip', 'scikit-learn']
pip dep:  ['azureml', 'azureml-contrib-dataset', 'azureml-contrib-services', 'azureml-core', 'azureml-defaults', 'azureml-mlflow', 'inference-schema', 'mlflow', 'pip', 'pymssql', 'scikit-learn-intelex', 'sqlalchemy']


{'_conda_dependencies': ordereddict([('name', 'project_environment'), ('dependencies', ['python=3.9.7', ordereddict([('pip', ['azureml', 'azureml-contrib-dataset', 'azureml-contrib-services', 'azureml-core', 'azureml-defaults', 'azureml-mlflow', 'inference-schema', 'mlflow', 'pip', 'pymssql', 'scikit-learn-intelex', 'sqlalchemy', 'https://nrosevearework0157530426.blob.core.windows.net/azureml/Environment/azureml-private-packages/DataScienceTools-0.0.1-py2.py3-none-any.whl'])]), 'matplotlib', 'numpy', 'packaging', 'pandas', 'pip', 'scikit-learn']), ('channels', ['anaconda', 'conda-forge'])]),
 '_python_version': '3.9.7'}

In [26]:
azure_env = core.Environment.get(ws, 'test')  #yaml_config['env_name'])  #, version=None, label=None)

conda_dep_check = azure_env.python.conda_dependencies.as_dict()
conda_dep_dict = conda_dependencies.as_dict()
conda_deps = set([dep for dep in conda_dep_check['dependencies'] 
    if not(isinstance(dep,dict))]), set([
        dep for dep in conda_dep_dict['dependencies']
            if not(isinstance(dep,dict))])

pip_deps = \
    set(chain.from_iterable(
        [dep['pip'] for dep in conda_dep_check['dependencies'] 
        if (isinstance(dep,dict) and ('pip' in dep))])), \
    set(chain.from_iterable(
        [dep['pip'] for dep in conda_dep_dict['dependencies'] 
            if (isinstance(dep,dict) and ('pip' in dep))]))
(conda_dep_dict['name'] == conda_dep_check['name'] and
                        (len(conda_deps[0].symmetric_difference(conda_deps[1])) == 0) and
                        (len(pip_deps[0].symmetric_difference(pip_deps[1])) == 0))

True

In [17]:
azure_env = core.Environment('test')
azure_env.python.conda_dependencies = conda_dependencies

if (ws is not None): # and self.register_env:
    azure_env.register(ws)

In [27]:
core.conda_dependencies.CondaDependencies.create?